In [1]:
#!pip install tweepy

In [2]:
#!pip install textblob 

In [3]:
#!pip install geopy

In [4]:
import tweepy
import socket
import re
from textblob import TextBlob
from geopy.geocoders import Nominatim
import json

In [5]:
ACCESS_TOKEN = '2664568128-XEpKbApevnp2NSfMrnPOi4es1hMvH4zjMz3VRf9'
ACCESS_SECRET = 'WoXOH7JdxJVhDzSZOzcBPD55IqWealhczbBohE1KUw18p'
CONSUMER_KEY = 'Wgy3Lnh3OmUSNbFG6ZedkrsZU'
CONSUMER_SECRET = 'vebIB0KXLNXwTTOU6OEjkCgSxfOYTEuuqT2HU6ic3LKKMbozAA'

In [6]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

In [7]:
hashtag = '#covid19'

In [8]:
TCP_IP = 'localhost'
TCP_PORT = 9001

In [9]:
geolocator = Nominatim(user_agent="location_test")

In [10]:
location = geolocator.geocode("Dallas")

In [11]:
data = {}
data['tweets'] = []

In [12]:
# this will get the location of the tweet using geopy
def get_tweet_loc(raw_location):
    location = geolocator.geocode(raw_location)
    return location

In [13]:
def get_tweet_sentiment(tweet):
    text = TextBlob(tweet)
    return text.sentiment

In [14]:
def preprocessing(tweet):
    
    # this will remove most all emoji characters from a given text 
    emojis = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f" 
        u"\u3030"
                           "]+", flags=re.UNICODE)
        
    tweet = re.sub(emojis, '', tweet)
    tweet = re.sub(r'/^\w+$/', '', tweet)
    return tweet



In [15]:
def getTweet(status):
    tweet = ""
    location = ""

    location = status.user.location
    
    if hasattr(status, "retweeted_status"):  # Check if Retweet
        try:
            tweet = status.retweeted_status.extended_tweet["full_text"]
        except AttributeError:
            tweet = status.retweeted_status.text
    else:
        try:
            tweet = status.extended_tweet["full_text"]
        except AttributeError:
            tweet = status.text

    return location, preprocessing(tweet)

In [16]:
# create sockets
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))

In [17]:
s.listen(1)

In [18]:
conn, addr = s.accept()

In [19]:
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        location, tweet = getTweet(status)
        
        loc_obj = get_tweet_loc(location)
        
        if (location != None and tweet != None and loc_obj != None):
            tweetLocation = location + "::" + tweet+"\n"

            display(tweetLocation)
            display(get_tweet_sentiment(tweet))
            display(loc_obj.address)
            
            conn.send(tweetLocation.encode('utf-8'))
            
            data['tweets'].append({
                'text': tweet,
                'polarity' : get_tweet_sentiment(tweet).polarity,
                'subjectivity' : get_tweet_sentiment(tweet).subjectivity,
                'rawLocation': location,
                'lat' : loc_obj.latitude,
                'long' : loc_obj.longitude,
                'address' : loc_obj.address
            })

        return True


    def on_error(self, status_code):
        if status_code == 420:
            return False
        else:
            display(status_code)

In [20]:
myStream = tweepy.Stream(auth=auth, listener=MyStreamListener())
myStream.filter(track=[hashtag], languages=["en"])

"USA ::Get ' em now!! https://t.co/iCdupg0spE\n"

Sentiment(polarity=0.0, subjectivity=0.0)

'United States'

"Atlanta, GA::This is a tragedy. It's actually a story about love so strong that even death couldn't keep them apart. I'm honored to be able to tell Willard Daniel Bowen and Wilma Gail Bowen's story. @FOX5Atlanta #COVID19 https://t.co/49ws77dcqU\n"

Sentiment(polarity=0.35833333333333334, subjectivity=0.5145833333333333)

...

'Northern Lights Avenue, Houghtonville, North Adams, Berkshire County, Massachusetts, 01247, United States of America'

'California, USA::WALL STREET BANKERS ARE NOT MORE “ESSENTIAL” THAN TEACHERS &amp; 1ST RESPONDERS—American Bankers Association is petitioning to designate Wall St finance workers as essential workers so they can get #COVID19 vaccines before most Americans. Cant make this up! https://t.co/lUyaWnjheh\n'

Sentiment(polarity=0.09375, subjectivity=0.4)

'California, United States of America'

KeyboardInterrupt: 

In [21]:
with open('data.txt', 'w') as outfile:
    json.dump(data['tweets'], outfile)